In [1]:
import numpy as np
import pandas as pd

In [2]:
#Load data
test = pd.DataFrame.from_csv("./rawdata/test_0.txt", sep="\t", header=None)
big_table = np.genfromtxt('./tables/svdtableProjection.txt', delimiter='\t')

In [3]:
big_table[big_table>5] = 5
big_table[big_table<0] = 1

In [4]:
big_table.shape

(200000, 216)

In [5]:
big_table[0][:20]

array([  4.25211973e+00,   1.00000000e+00,   1.00000000e+00,
         3.81877528e-16,   1.00000000e+00,   1.00000000e+00,
         1.61997918e-17,   1.00000000e+00,   3.58087538e-17,
         1.00000000e+00,   3.00574181e-18,   2.19995831e-17,
         1.00000000e+00,   8.74372634e-18,   4.13603231e-02,
         1.00000000e+00,   5.02486971e-17,   1.00000000e+00,
         1.00000000e+00,   1.00000000e+00])

In [6]:
def CaluateScore(user, genre, table):
    return table[user][genre]

In [7]:
test = test.reset_index()
new_name = dict(zip([0,1,2], ["user","song_id","rating"]))
test = test.rename(columns=new_name)

In [8]:
test.head()

,user,song_id,rating
0,0,7171,5
1,0,8637,4
2,0,21966,4
3,0,35821,5
4,0,82446,5


In [9]:
song_att_raw  = pd.DataFrame.from_csv("./rawdata/song-attributes.txt", sep="\t", header=None)
origin = [0,1,2,3]
modified1 = ["song_id","album_id", "artist_id", "genre_id"]
new_name = dict(zip(origin, modified1))
song_att = song_att_raw.rename(columns=new_name)
song_att.head()

,album_id,artist_id,genre_id
0,,,
0,12070,8490,0
1,19512,7975,134
2,18953,3492,0
3,695,2653,0
4,243,2282,0


In [10]:
result_all = test.join(song_att, on="song_id", how="right")
result_all.head()

,user,song_id,rating,album_id,artist_id,genre_id
0,0.0,7171,5.0,19948,2889,0
43791,4379.0,7171,2.0,19948,2889,0
134420,13442.0,7171,5.0,19948,2889,0
139461,13946.0,7171,5.0,19948,2889,0
172151,17215.0,7171,5.0,19948,2889,0


In [29]:
len(result_all)

2008965

In [30]:
result_all = result_all.dropna()

In [62]:
result = result_all.sample(40000)

In [63]:
result.head()

,user,song_id,rating,album_id,artist_id,genre_id
1724311,172431.0,10433,1.0,12109,4681,0
14379,1437.0,135789,4.0,12941,148,0
893799,89379.0,135205,4.0,16075,5797,134
919929,91992.0,91863,3.0,18555,1682,0
1654404,165440.0,50215,5.0,15143,2988,0


In [64]:
answer = list(map(lambda x: CaluateScore(result['user'][x].astype(int), result['genre_id'][x].astype(int), big_table), result.index))

In [65]:
len(answer)

40000

In [69]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [70]:
print("MSE:",mean_squared_error(result['rating'], answer))
print("MAE:",mean_absolute_error(result['rating'], answer))

MSE: 1.78508346268
MAE: 1.03862503871


In [37]:
resultun = result_all[result_all['genre_id']!=0]

In [38]:
resultun.head()

,user,song_id,rating,album_id,artist_id,genre_id
16,1.0,62770,3.0,17769,5955,106
813,81.0,62770,3.0,17769,5955,106
14176,1417.0,62770,2.0,17769,5955,106
39715,3971.0,62770,5.0,17769,5955,106
56694,5669.0,62770,4.0,17769,5955,106


In [39]:
len(resultun)

250281

In [40]:
resultun = resultun.sample(20000)

In [41]:
answerun = list(map(lambda x: CaluateScore(int(resultun['user'][x]), int(resultun['genre_id'][x]), big_table), resultun.index))

In [42]:
len(answerun)

20000

In [71]:
print("MSE:",mean_squared_error(resultun['rating'], answerun))
print("MAE:",mean_absolute_error(resultun['rating'], answerun))
#mean_squared_error(resultun['rating'], answerun)

MSE: 2.69364960734
MAE: 1.16421426263
